In [2]:
pip install beautifulsoup4

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

## 1. Data from unstructured websites

In [4]:
url = "https://tastesbetterfromscratch.com/meal-plan-195"
headers ={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)" }
req = requests.Session()
response = req.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
# initialize an empty list
rows = []

# iterate over all rows in the faculty table
for meal in soup.find_all("p", {"class": "has-text-align-left"})[0:]:

    day = meal.text.strip()

    name_link_tag = meal.find("a")
    name_link = name_link_tag['href'] if name_link_tag and 'href' in name_link_tag.attrs else None
    name = name_link_tag.text.strip() if name_link_tag else None

    price = None
    price_match = re.search(r'\$(\d+\.?\d*)', meal.text)
    if price_match:
        price = price_match.group(0)

    rows.append({
        "day": day,
        "name": name,
        "link": name_link,
        "price": price
    })

In [6]:
pd.DataFrame(rows)

,day,name,link,price
0,Monday: Lasagna $20.02,Lasagna,https://tastesbetterfromscratch.com/classic-it...,$20.02
1,Tuesday: Broccoli Cheese Soup $13.48,Broccoli Cheese Soup,https://tastesbetterfromscratch.com/broccoli-c...,$13.48
2,Wednesday: Kung Pao Chicken $13.86,Kung Pao Chicken,https://tastesbetterfromscratch.com/restaurant...,$13.86
3,Thursday: Avocado Chicken Salad Lettuce Wraps ...,Avocado Chicken Salad Lettuce Wraps,https://tastesbetterfromscratch.com/avocado-ch...,$13.42
4,Friday: Mexican Pizza $17.51,Mexican Pizza,https://tastesbetterfromscratch.com/mexican-pi...,$17.51


## 2. Data from an API

In [ ]:
url = "https://tasty.p.rapidapi.com/recipes/list"

querystring = {"from":"0","size":"100","q":"lasagna"}

headers = {
    "X-RapidAPI-Key": "9cd3c50ad0msh8dc3e7c2d08067bp1d03a5jsn2dfc3305943d",
    "X-RapidAPI-Host": "tasty.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

In [8]:
lasagna_recipes = pd.json_normalize(response.json(), "results")
lasagna_recipes.head()

,approved_at,aspect_ratio,beauty_url,brand,brand_id,buzz_id,canonical_id,compilations,cook_time_minutes,country,created_at,credits,description,draft_status,facebook_posts,id,inspired_by_url,instructions,is_app_only,is_one_top,is_shoppable,is_subscriber_content,keywords,language,name,num_servings,nutrition_visibility,original_video_url,prep_time_minutes,promotion,renditions,sections,seo_path,seo_title,servings_noun_plural,servings_noun_singular,show_id,slug,tags,thumbnail_alt_text,thumbnail_url,tips_and_ratings_enabled,topics,total_time_minutes,total_time_tier,updated_at,video_ad_content,video_id,video_url,yields,nutrition.calories,nutrition.carbohydrates,nutrition.fat,nutrition.fiber,nutrition.protein,nutrition.sugar,nutrition.updated_at,price.consumption_portion,price.consumption_total,price.portion,price.total,price.updated_at,show.id,show.name,tips_summary.by_line,tips_summary.content,tips_summary.header,user_ratings.count_negative,user_ratings.count_positive,user_ratings.score,total_time_tier.display_tier,total_time_tier.tier,brand.id,brand.image_url,brand.name,brand.slug
0,1563370544,None,None,NaN,NaN,NaN,recipe:4885,[],0,US,1553790152,"[{'is_verified': False, 'name': None, 'picture...",Dive into layers of savory goodness with this ...,published,[],4885,None,"[{'appliance': 'oven', 'display_text': 'Prehea...",False,False,True,False,,eng,Savory Lasagna,12,auto,None,0,partial,[],"[{'components': [{'extra_comment': '', 'id': 5...","9295813,64486,9299522,9299622",,servings,serving,17,savoury-lasagna,"[{'display_name': 'Italian', 'id': 64453, 'nam...",,https://img.buzzfeed.com/tasty-app-user-assets...,True,"[{'name': 'Easy Dinner', 'slug': 'easy-dinner'...",0,NaN,1683237600,None,NaN,None,12 Servings,394.0,22.0,21.0,37.0,24.0,7.0,2024-05-01T08:05:37+02:00,100,1450,200,2600,2025-10-24T07:15:41+02:00,17,Tasty,Powered By Botatouille,• Reduce the amount of sauce to 4 cups for a d...,Highlights,10,148,0.936709,NaN,NaN,NaN,NaN,NaN,NaN
1,1500676550,1:1,None,NaN,NaN,NaN,recipe:1180,"[{'approved_at': 1578228259, 'aspect_ratio': '...",45,US,1498771623,"[{'is_verified': False, 'name': 'Claire Nolan'...","This unique spin on lasagna is a cheesy, indul...",published,[],1180,None,"[{'appliance': 'oven', 'display_text': 'Prehea...",False,False,True,False,"alfredo, alfredo lasagna, broccoli, buzzfeed, ...",eng,Broccoli Chicken Alfredo Lasagna,6,auto,https://s3.amazonaws.com/video-api-prod/assets...,30,full,"[{'aspect': 'square', 'bit_rate': 7336, 'conta...","[{'components': [{'extra_comment': '', 'id': 1...","8757513,9295874,64453",None,servings,serving,27,broccoli-chicken-alfredo-lasagna,"[{'display_name': 'Italian', 'id': 64453, 'nam...",,https://img.buzzfeed.com/video-api-prod/assets...,True,"[{'name': 'Easy Dinner', 'slug': 'easy-dinner'...",75,NaN,1683237600,none,16936.0,https://vid.tasty.co/output/33580/low_14968690...,Servings: 6-8,687.0,17.0,56.0,2.0,34.0,4.0,2024-09-03T03:10:44+02:00,200,1350,350,2150,2025-10-24T07:10:52+02:00,27,Tasty: Tasty Junior,Powered By Botatouille,• Double the Alfredo sauce for a creamier expe...,Highlights,35,724,0.953887,Under 1.5 hours,under_1.5_hours,NaN,NaN,NaN,NaN
2,1576941907,1:1,None,NaN,NaN,NaN,recipe:5943,"[{'approved_at': 1603460823, 'aspect_ratio': '...",30,US,1576886307,"[{'is_verified': False, 'name': 'Betsy Carter'...",When layers of cheesy lasagna come together fl...,published,[],5943,None,"[{'appliance': 'oven', 'display_text': 'Prehea...",False,False,True,False,"adulting, baking recipes, basics of cooking, b...",eng,Classic Lasagna,6,auto,https://s3.amazonaws.com/video-api-prod/assets...,20,full,"[{'aspect': 'square', 'bit_rate': 1912, 'conta...",[{'components': [{'extra_comment': 'for greasi...,"9295813,64486,9299522,9299621",,servings,serving,17,classic-lasagna,"[{'display_name': 'Italian', 'id': 64453, 'nam...",,https://img.buzzfeed.com/thumbnailer-prod-us-e...,True,"[{'name': 'Easy Dinner', 'slug': 'easy-dinner'...",50,NaN,1576882800,none,98140.0,https://vid.tasty.co/output/156136/hls24_1

## 3. Automate it

In [9]:
def get_mealplan_data(meal_num):
  url = f"https://tastesbetterfromscratch.com/meal-plan-{meal_num}"
  headers ={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)" }
  req = requests.Session()
  response = req.get(url, headers=headers)
  soup = BeautifulSoup(response.content, "html.parser")

  # initialize an empty list
  rows = []

  # iterate over all rows in the faculty table
  for meal in soup.find_all("p", {"class": "has-text-align-left"})[0:]:
    day = meal.text.strip()
    name_link_tag = meal.find("a")
    name = name_link_tag.text.strip() if name_link_tag else None
    rows.append({
        "day": day,
        "name": name
    })

  item = rows[0]["name"]
  url2 = "https://tasty.p.rapidapi.com/recipes/list"
  querystring = {"from":"0","size":"5","q":item}

  headers = {
    "X-RapidAPI-Key": "9cd3c50ad0msh8dc3e7c2d08067bp1d03a5jsn2dfc3305943d",
    "X-RapidAPI-Host": "tasty.p.rapidapi.com"
  }

  response = requests.get(url2, headers=headers, params=querystring)
  recipes = pd.json_normalize(response.json(), "results")
  return recipes.head()

In [10]:
df = get_mealplan_data(202)
df

""


## 4. Add a column with fuzzy matching

In [ ]:
df["Vegetarian"] = df["name"].apply(lambda x: 1 if "chicken" or "beef" or "pork" not in x.lower() else 0)